# Portfolio Optimization

This file is used to develop an example for stock optimization model

---

In [1]:
# import packages
import pandas as pd
import boto3
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import cvxpy as cp
import warnings
warnings.filterwarnings("ignore")

In [2]:
# load file
df = pd.read_csv('../data/stock_data_2023-01-01_to_2025-06-29.csv')
df.head()

,Date,Type,Ticker,Price
0,2023-01-03,Close,A,1.476176e+02
1,2023-01-03,High,A,1.506578e+02
2,2023-01-03,Low,A,1.460730e+02
3,2023-01-03,Open,A,1.495066e+02
4,2023-01-03,Volume,A,1.414300e+06


---
## 1. Modern Portfolio Theory (MPT)

**Idea:** combine assets with low correlation to each other to avoid significant losses shoudl one asset underperforms.

*Def:* 
- Returns represent the gains or losses generated by these assets over a specific time period, typically expressed as a percentage.
- Risk refers to the uncertainty or volatility of returns. It includes market fluctuations, economic conditions, and company-specific events.

In [3]:
# Get some example tickers for testing
tickers = ['META', 'AAPL', 'AMZN', 'NFLX', 'WMT', 'MSFT', 'GOOGL', 'TSLA']

# Filter the data based on those tickers
df_filtered = df.loc[(df['Ticker'].isin(tickers)) & (df['Type'] == 'Close')]

In [4]:
# Pivot the table
df_filtered['Date'] = pd.to_datetime(df_filtered['Date'])
pivot_df = df_filtered.pivot(index = 'Date', columns = 'Ticker', values = 'Price').sort_index()

In [5]:
# Infer average # of trading days per year
date_range_days = (pivot_df.index[-1] - pivot_df.index[0]).days
total_obs = len(pivot_df)
trading_days_per_year = round(total_obs / (date_range_days / 365))

print(f"Inferred trading days/year: {trading_days_per_year}")

Inferred trading days/year: 251


In [6]:
# Compute log return, mean, and variance
log_ret = np.log(pivot_df / pivot_df.shift(1)).dropna()

mu = log_ret.mean() * trading_days_per_year                 # Annualized expected returns
Sigma = log_ret.cov() * trading_days_per_year               # Annualized covariance matrix

In [7]:
# Solve Minimum-Variance Portfolio
n = len(tickers)
w = cp.Variable(n)

# Objective: minimize risk
objective = cp.Minimize(cp.quad_form(w, Sigma.values))

# constraint
constraints = [
    cp.sum(w) == 1,
    w >= 0
]

# Solve the problem
problem = cp.Problem(objective, constraints)
problem.solve()

opt_w = w.value.round(4)

In [8]:
capital = 15_000
latest_prices = pivot_df.iloc[-1]

alloc_usd = capital * opt_w
alloc_shares = (alloc_usd / latest_prices).round(2)

result = pd.DataFrame({
    'Weight': opt_w,
    'Dollars': alloc_usd.round(2),
    'Shares': alloc_shares
}, index=tickers)

result

,Weight,Dollars,Shares
META,0.0906,1359.0,0.00
AAPL,0.0000,0.0,6.76
AMZN,0.0797,1195.5,0.00
NFLX,0.0000,0.0,0.49
WMT,0.2225,3337.5,86.96
MSFT,0.0432,648.0,6.73
GOOGL,0.0000,0.0,6.70
TSLA,0.5639,8458.5,0.00


In [9]:
expected_return = float(mu.values @ opt_w)
volatility = float(np.sqrt(opt_w @ Sigma.values @ opt_w))
rf = 0.04
sharpe_ratio = (expected_return - rf) / volatility
print("\nPortfolio stats:")
print(f"  Expected return: {expected_return:.2%}")
print(f"  Volatility     : {volatility:.2%}")
print(f"  Sharpe ratio   : {sharpe_ratio:.2f}")



Portfolio stats:
  Expected return: 30.23%
  Volatility     : 16.38%
  Sharpe ratio   : 1.60
